# Render a HyperNeRF video!

**Author**: [Keunhong Park](https://keunhong.com)

[[Project Page](https://hypernerf.github.io)]
[[Paper](https://arxiv.org/abs/2106.13228)]
[[GitHub](https://github.com/google/hypernerf)]

This notebook renders a video using the test cameras generated in the capture processing notebook.

You can also load your own custom cameras by modifying the code slightly.

### Instructions

1. Convert a video into our dataset format using the [capture processing notebook](https://colab.sandbox.google.com/github/google/nerfies/blob/main/notebooks/Nerfies_Capture_Processing.ipynb).
2. Train a HyperNeRF model using the [training notebook](https://colab.sandbox.google.com/github/google/hypernerf/blob/main/notebooks/HyperNeRF_Training.ipynb)
3. Run this notebook!


### Notes
 * Please report issues on the [GitHub issue tracker](https://github.com/google/hypernerf/issues).

## Environment Setup

In [ ]:
# !pip install flax immutabledict mediapy
# !pip install git+https://github.com/google/hypernerf

In [ ]:
# @title Configure notebook runtime
# @markdown If you would like to use a GPU runtime instead, change the runtime type by going to `Runtime > Change runtime type`. 
# @markdown You will have to use a smaller batch size on GPU.

import jax

runtime_type = 'gpu'  # @param ['gpu', 'tpu']
if runtime_type == 'tpu':
  import jax.tools.colab_tpu
  jax.tools.colab_tpu.setup_tpu()

print('Detected Devices:', jax.devices())

In [ ]:
# # @title Mount Google Drive
# # @markdown Mount Google Drive onto `/content/gdrive`. You can skip this if running locally.

# from google.colab import drive
# drive.mount('/content/gdrive')

In [ ]:
# @title Define imports and utility functions.

import jax
from jax.config import config as jax_config
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random

import flax
import flax.linen as nn
from flax import jax_utils
from flax import optim
from flax.metrics import tensorboard
from flax.training import checkpoints

from absl import logging
from io import BytesIO
import random as pyrandom
import numpy as np
import PIL
import IPython
import tempfile
import imageio
import mediapy
from IPython.display import display, HTML
from base64 import b64encode


# Monkey patch logging.
def myprint(msg, *args, **kwargs):
 print(msg % args)

logging.info = myprint 
logging.warn = myprint
logging.error = myprint

In [ ]:
# @title Model and dataset configuration
# @markdown Change the directories to where you saved your capture and experiment.

import os

import sys
if os.path.exists(os.path.expanduser('~/hypernerf-barf/')):
    project_root = os.path.expanduser('~/hypernerf-barf/')
elif os.path.exists(os.path.expanduser('~/3d_cv/repos/hypernerf_barf/')):
    project_root = os.path.expanduser('~/3d_cv/repos/hypernerf_barf/')
else:
    raise NotImplemented
sys.path.insert(0, project_root)

if os.path.exists('/ssd/zhiwen/data/hypernerf/raw/'):
    data_root = '/ssd/zhiwen/data/hypernerf/raw/'
elif os.path.exists('/hdd/zhiwen/data/hypernerf/raw/'):
    data_root = '/hdd/zhiwen/data/hypernerf/raw/'
elif os.path.exists('/home/zwyan/3d_cv/data/hypernerf/raw/'):
    data_root = '/home/zwyan/3d_cv/data/hypernerf/raw/'
else:
    raise NotImplemented

from pathlib import Path
from pprint import pprint
import gin
from IPython.display import display, Markdown

from hypernerf import models
from hypernerf import modules
from hypernerf import warping
from hypernerf import datasets
from hypernerf import configs


# @markdown The working directory where the trained model is.
# train_dir = os.path.join(project_root, 'experiments/as7_q_base_exp01')  # @param {type: "string"}
# train_dir = os.path.join(project_root, 'experiments/wb5_test_base')  # @param {type: "string"}
# train_dir = os.path.join(project_root, 'experiments/wb5_hcxt_exp01')  # @param {type: "string"}
# train_dir = os.path.join(project_root, 'experiments/b2_q_hs_exp04')  # @param {type: "string"}
train_dir = os.path.join(project_root, 'experiments/b2_q_hsf_exp04')  # @param {type: "string"}

# @markdown The directory to the dataset capture.
# data_dir = os.path.join(data_root, 'americano/')  # @param {type: "string"}
# data_dir = os.path.join(data_root, 'white-board-5/')  # @param {type: "string"}
# data_dir = os.path.join(data_root, 'aluminium-sheet-6/')  # @param {type: "string"}
# data_dir = os.path.join(data_root, 'aluminium-sheet-7_qualitative/')  # @param {type: "string"}
# data_dir = os.path.join(data_root, 'collect-americano/')  # @param {type: "string"}
# data_dir = os.path.join(data_root, 'vrig-chicken/')  # @param {type: "string"}
# data_dir = os.path.join(data_root, 'bell-2_qualitative/')  # @param {type: "string"}
data_dir = os.path.join(data_root, 'bell-2_qualitative/')  # @param {type: "string"}

checkpoint_dir = Path(train_dir, 'checkpoints')
checkpoint_dir.mkdir(exist_ok=True, parents=True)

config_path = Path(train_dir, 'config.gin')
with open(config_path, 'r') as f:
  logging.info('Loading config from %s', config_path)
  config_str = f.read()
gin.parse_config(config_str)

config_path = Path(train_dir, 'config.gin')
with open(config_path, 'w') as f:
  logging.info('Saving config to %s', config_path)
  f.write(config_str)
    
exp_config = configs.ExperimentConfig()
train_config = configs.TrainConfig()
eval_config = configs.EvalConfig()
spec_config = configs.SpecularConfig()

eval_config.chunk = 2048

display(Markdown(
    gin.config.markdown(gin.config_str())))

In [ ]:
# @title Create datasource and show an example.

from hypernerf import datasets
from hypernerf import image_utils

if spec_config.use_hyper_spec_model:
    dummy_model = models.HyperSpecModel({}, 0, 0)
else:
    dummy_model = models.NerfModel({}, 0, 0)
datasource = exp_config.datasource_cls(
    data_dir=data_dir,
    image_scale=exp_config.image_scale,
    random_seed=exp_config.random_seed,
    # Enable metadata based on model needs.
    use_warp_id=dummy_model.use_warp,
    use_appearance_id=(
        dummy_model.nerf_embed_key == 'appearance'
        or dummy_model.hyper_embed_key == 'appearance'),
    use_camera_id=dummy_model.nerf_embed_key == 'camera',
    use_time=dummy_model.warp_embed_key == 'time')

mediapy.show_image(datasource.load_rgb(datasource.train_ids[0]))

In [ ]:
# @title Load model
# @markdown Defines the model and initializes its parameters.

from flax.training import checkpoints
from hypernerf import models
from hypernerf import model_utils
from hypernerf import schedules
from hypernerf import training

rng = random.PRNGKey(exp_config.random_seed)
np.random.seed(exp_config.random_seed + jax.process_index())
devices_to_use = jax.devices()

learning_rate_sched = schedules.from_config(train_config.lr_schedule)
nerf_alpha_sched = schedules.from_config(train_config.nerf_alpha_schedule)
warp_alpha_sched = schedules.from_config(train_config.warp_alpha_schedule)
elastic_loss_weight_sched = schedules.from_config(
train_config.elastic_loss_weight_schedule)
hyper_alpha_sched = schedules.from_config(train_config.hyper_alpha_schedule)
hyper_sheet_alpha_sched = schedules.from_config(
    train_config.hyper_sheet_alpha_schedule)
norm_loss_weight_sched = schedules.from_config(spec_config.norm_loss_weight_schedule)
norm_input_alpha_sched = schedules.from_config(spec_config.norm_input_alpha_schedule)
norm_voxel_lr_sched = schedules.from_config(spec_config.norm_voxel_lr_schedule)
norm_voxel_ratio_sched = schedules.from_config(spec_config.norm_voxel_ratio_schedule)

rng, key = random.split(rng)
params = {}
model, params['model'] = models.construct_nerf(
    key,
    use_hyper_spec_model=spec_config.use_hyper_spec_model,
    batch_size=train_config.batch_size,
    embeddings_dict=datasource.embeddings_dict,
    near=datasource.near,
    far=datasource.far,
    screw_input_mode=spec_config.screw_input_mode,
    use_sigma_gradient=spec_config.use_sigma_gradient,
    use_predicted_norm=spec_config.use_predicted_norm,
)

optimizer_def = optim.Adam(learning_rate_sched(0))
if model.use_flow_model:
    focus = flax.traverse_util.ModelParamTraversal(lambda p, _: 'flow_model' not in p)
    optimizer = optimizer_def.create(params, focus=focus)
else:
    optimizer = optimizer_def.create(params)

# state = model_utils.TrainState(optimizer=optimizer)
state = model_utils.TrainState(
    optimizer=optimizer,
    nerf_alpha=nerf_alpha_sched(0),
    warp_alpha=warp_alpha_sched(0),
    hyper_alpha=hyper_alpha_sched(0),
    hyper_sheet_alpha=hyper_sheet_alpha_sched(0),
    norm_loss_weight=norm_loss_weight_sched(0),
    norm_input_alpha=norm_input_alpha_sched(0),
    norm_voxel_lr=norm_voxel_lr_sched(0),
    norm_voxel_ratio=norm_voxel_ratio_sched(0),
)
scalar_params = training.ScalarParams(
  learning_rate=learning_rate_sched(0),
  elastic_loss_weight=elastic_loss_weight_sched(0),
  warp_reg_loss_weight=train_config.warp_reg_loss_weight,
  warp_reg_loss_alpha=train_config.warp_reg_loss_alpha,
  warp_reg_loss_scale=train_config.warp_reg_loss_scale,
  background_loss_weight=train_config.background_loss_weight,
  hyper_reg_loss_weight=train_config.hyper_reg_loss_weight,
  sigma_grad_diff_reg_weight=spec_config.sigma_grad_diff_reg_weight,
  back_facing_reg_weight=spec_config.back_facing_reg_weight,
  hyper_concentration_reg_weight=spec_config.hyper_concentration_reg_weight,
  hyper_concentration_reg_scale=spec_config.hyper_concentration_reg_scale,
  hyper_jacobian_reg_weight=spec_config.hyper_jacobian_reg_weight,
  hyper_jacobian_reg_scale=spec_config.hyper_jacobian_reg_scale,
  hyper_c_jacobian_reg_weight=spec_config.hyper_c_jacobian_reg_weight,
  hyper_c_jacobian_reg_scale=spec_config.hyper_c_jacobian_reg_scale,
  norm_voxel_loss_weight=spec_config.norm_voxel_loss_weight,
)

logging.info('Restoring checkpoint from %s', checkpoint_dir)
state = checkpoints.restore_checkpoint(checkpoint_dir, state)
step = state.optimizer.state.step + 1
state = jax_utils.replicate(state, devices=devices_to_use)
del params

In [ ]:
# @title Define pmapped render function.

import functools
from hypernerf import evaluation

devices = jax.devices()


def _model_fn(key_0, key_1, key_2, params, rays_dict, extra_params):
  out = model.apply({'params': params},
                    rays_dict,
                    extra_params=extra_params,
                    rngs={
                        'coarse': key_0,
                        'fine': key_1,
                        'voxel': key_2
                    },
                    mutable=False,
                    screw_input_mode=spec_config.screw_input_mode,
                    use_sigma_gradient=spec_config.use_sigma_gradient,
                    use_predicted_norm=spec_config.use_predicted_norm,
                    return_points=False,
                    return_nv_details=False,
                    norm_voxel_ratio=1, # inference ratio is always 1
                   )
  return jax.lax.all_gather(out, axis_name='batch')

pmodel_fn = jax.pmap(
    # Note rng_keys are useless in eval mode since there's no randomness.
    _model_fn,
    in_axes=(0, 0, 0, 0, 0, 0),  # Only distribute the data input.
    devices=devices_to_use,
    axis_name='batch',
)

render_fn = functools.partial(evaluation.render_image,
                              model_fn=pmodel_fn,
                              device_count=len(devices),
                              chunk=eval_config.chunk)

In [ ]:
# @title Load cameras.

from hypernerf import utils

# camera_path = 'camera'
camera_path = 'fix_camera_93'  # @param {type: 'string'}
# camera_path = 'vrig_camera'  # @param {type: 'string'}
# camera_path = 'interpolate_camera'  # @param {type: 'string'}

camera_dir = Path(data_dir, camera_path)
print(f'Loading cameras from {camera_dir}')
test_camera_paths = datasource.glob_cameras(camera_dir)
test_cameras = utils.parallel_map(datasource.load_camera, test_camera_paths, show_pbar=True)

In [ ]:
# @title Render video frames.
from hypernerf import visualization as viz
from hypernerf import model_utils


rng = rng + jax.process_index()  # Make random seed separate across hosts.
keys = random.split(rng, len(devices))

results = []
raw_result_list = []
step_size = 9 if camera_path.startswith("vrig") else 10

for i in range(0, len(test_cameras), step_size):
# for i in range(330, 400):
  print(f'Rendering frame {i+1}/{len(test_cameras)}')
  camera = test_cameras[i]
  batch = datasets.camera_to_rays(camera)
  if not camera_path.startswith('vrig'):
    batch['metadata'] = {
        'appearance': jnp.ones_like(batch['origins'][..., 0, jnp.newaxis], jnp.uint32) * i,
        'warp': jnp.ones_like(batch['origins'][..., 0, jnp.newaxis], jnp.uint32) * i,
        'camera': jnp.ones_like(batch['origins'][..., 0, jnp.newaxis], jnp.uint32) * 0
    }
  else:
    batch['metadata'] = {
#         'appearance': jnp.ones_like(batch['origins'][..., 0, jnp.newaxis], jnp.uint32) * ((i + 1) % 2 + 1),
        'appearance': jnp.ones_like(batch['origins'][..., 0, jnp.newaxis], jnp.uint32) * i,
        'warp': jnp.ones_like(batch['origins'][..., 0, jnp.newaxis], jnp.uint32) * i,
        'camera': jnp.ones_like(batch['origins'][..., 0, jnp.newaxis], jnp.uint32) * ((i + 1) % 2 + 1)
    }

  render = render_fn(state, batch, rng=rng)
#   raw_result = {}
#   for key in render:
#     raw_result[key] = np.array(render[key])
#   raw_result_list.append(raw_result)

  rgb = np.array(render['rgb'])
  depth_med = np.array(render['med_depth'])

  dummy_image = np.zeros_like(rgb)

  sigma_gradient = np.array(render['ray_sigma_gradient'])
  sigma_gradient = model_utils.normalize_vector(sigma_gradient)
  sigma_gradient = sigma_gradient / 2.0 + 0.5

  sigma_gradient_r = np.array(render['ray_sigma_gradient_r'])
  sigma_gradient_r = sigma_gradient_r / 2.0 + 0.5
    
  if 'ray_inter_norm' in render:
      ray_inter_norm = np.array(render['ray_inter_norm'])
      ray_inter_norm = ray_inter_norm / 2.0 + 0.5
  else:
    ray_inter_norm = np.zeros_like(sigma_gradient)

  ray_rotation_field = np.array(render['ray_rotation_field'])
  ray_rotation_field = ray_rotation_field / 2.0 + 0.5
    
  ray_translation_field = np.array(render['ray_translation_field'])
  ray_translation_field = np.abs(ray_translation_field)
  print(np.max(ray_translation_field))
    
  ray_hyper_points = np.array(render['ray_hyper_points'])
  ray_hyper_points = np.abs(ray_hyper_points)
  if ray_hyper_points.shape[-1] > 0:
    ray_hyper_points = ray_hyper_points / np.max(ray_hyper_points)
    dummy_hyper = np.ones([ray_hyper_points.shape[0], ray_hyper_points.shape[1], 1]) * 0
    ray_hyper_points = np.concatenate([ray_hyper_points, dummy_hyper], axis=-1)
#   print(np.max(ray_hyper_points, axis=1))
  else:
    ray_hyper_points = dummy_image

  ray_hyper_c = np.array(render['ray_hyper_c'])
  ray_hyper_c = np.abs(ray_hyper_c)
  ray_hyper_c = ray_hyper_c / np.max(ray_hyper_c)
  if ray_hyper_c.shape[-1] == 2:
    dummy_hyper = np.ones([ray_hyper_c.shape[0], ray_hyper_points.shape[1], 1]) * 0
    dummy_hyper_c = np.ones([ray_hyper_c.shape[0], ray_hyper_c.shape[1], 1]) * 0
    ray_hyper_c = np.concatenate([ray_hyper_c, dummy_hyper], axis=-1)
  else:
    ray_hyper_c = ray_hyper_c[..., :3]
    
#   ray_s_weights = np.array(render['ray_s_weights'])
#   ray_s_weights = np.tile(ray_s_weights, [1, 1, 3])
    
#   ray_d_color = np.array(render['ray_d_color'])
#   ray_s_color = np.array(render['ray_s_color'])

  if 'ray_color_delta_x' in render:
    ray_color_delta_x = np.array(render['ray_color_delta_x'])
    ray_color_delta_x = (ray_color_delta_x - np.min(ray_color_delta_x)) * 100
  else:
    ray_color_delta_x = dummy_image

  results.append((rgb, depth_med, sigma_gradient, ray_inter_norm, 
                  ray_color_delta_x, ray_translation_field, ray_hyper_points, ray_hyper_c, dummy_image))
  depth_viz = viz.colorize(depth_med.squeeze(), cmin=datasource.near, cmax=datasource.far, invert=True)
  mediapy.show_images([rgb, depth_viz, sigma_gradient, ray_inter_norm, 
                       ray_color_delta_x, ray_translation_field, ray_hyper_points, ray_hyper_c], columns=4)

In [ ]:
# @title Show rendered video.

fps = 30  # @param {type:'number'}

frames = []
for rgb, depth, sigma_gradient, ray_inter_norm, ray_rotation_field, ray_translation_field, ray_hyper_points, ray_hyper_c, dummy_image in results:
  depth_viz = viz.colorize(depth.squeeze(), cmin=datasource.near, cmax=datasource.far, invert=True)
#   frame = np.concatenate([rgb, depth_viz, sigma_gradient, sigma_gradient_r, 
#                           ray_rotation_field, ray_hyper_points, ray_hyper_c], axis=1)
  row1 = np.concatenate([rgb, depth_viz, sigma_gradient, ray_inter_norm], axis=1)
  row2 = np.concatenate([ray_rotation_field, ray_translation_field, ray_hyper_points, ray_hyper_c], axis=1)
  frame = np.concatenate([row1, row2], axis=0)
  frames.append(image_utils.image_to_uint8(frame))

mediapy.set_show_save_dir(train_dir)
mediapy.show_video(frames, fps=fps, title="result_{}".format(camera_path))

In [ ]:
raw_result_save_path = os.path.join(train_dir, "render_result_{}".format(camera_path))
with open(raw_result_save_path, "wb+") as f:
    np.save(f, raw_result_list)

In [ ]:
import matplotlib.pyplot as plt
from jax import numpy as jnp

import os

import sys
if os.path.exists(os.path.expanduser('~/hypernerf-barf/')):
    project_root = os.path.expanduser('~/hypernerf-barf/')
elif os.path.exists(os.path.expanduser('~/3d_cv/repos/hypernerf_barf/')):
    project_root = os.path.expanduser('~/3d_cv/repos/hypernerf_barf/')
else:
    raise NotImplemented
sys.path.insert(0, project_root)
from hypernerf import utils

hyper_points = jnp.array([0, -0.05, -0.1, -0.3, -0.4, -0.5, -0.8, -1.2])
hyper_points = jnp.arange(-5, 5, 0.1)
# hyper_concentration_reg_loss = utils.general_loss_with_squared_residual(
# hyper_points, alpha=-2, scale=0.1
# )
hyper_concentration_reg_loss = utils.gm_loss(hyper_points, scale=0.1)
# print(hyper_concentration_reg_loss)
# print(spec_config.hyper_concentration_reg_scale)
plt.plot(hyper_points, hyper_concentration_reg_loss)
plt.show()

In [ ]:
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")